In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd


KeyboardInterrupt



In [2]:
#Open bio data
bio_states_2019 = xr.open_dataset("/cluster/projects/itk-SINMOD/coral-mapping/midnor/BioStates_2019.nc")


In [3]:
print(list(bio_states_2019.variables))

['time', 'grid_mapping', 'LayerDepths', 'xc', 'yc', 'zc', 'depth', 'DXxDYy', 'nitrate', 'silicate', 'ammonium', 'diatoms', 'flagellates', 'ciliates', 'HNANO', 'bacteria', 'calanus_finmarchicus', 'calanus_glacialis', 'detritus_slow', 'detritus_fast', 'DOC', 'cDOM', 'silicate_detritus', 'sediment_Si', 'sediment_N']


In [16]:
diatoms = bio_states_2019["diatoms"]
flagellates = bio_states_2019["flagellates"]

chlorophyll = 3.192*diatoms + 1.186*flagellates    #[mg/m3]

In [17]:
nitrate = bio_states_2019["nitrate"]
silicate = bio_states_2019["silicate"]

In [18]:
#Zooplankton
calanus_finmarchicus = bio_states_2019["calanus_finmarchicus"]
calanus_glacialis = bio_states_2019["calanus_glacialis"]

In [19]:
bacteria = bio_states_2019["bacteria"]

In [ ]:
 #calcite and aragonite saturation state??

In [11]:
#downwelling

In [ ]:
#upwelling


In [20]:

phys_states_2019 = xr.open_dataset("/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc")


In [21]:
print(list(phys_states_2019.variables))

['time', 'grid_mapping', 'LayerDepths', 'xc', 'yc', 'zc', 'depth', 'DXxDYy', 'u_velocity', 'v_velocity', 'elevation', 'temperature', 'salinity', 'ice_thickness', 'ice_compactness', 'salinity_ice']


In [24]:
coral_data = pd.read_parquet('/cluster/home/maikents/midnor_total_coral_data_processed_v2.parquet')

In [26]:
print(list(coral_data))
coral_x = coral_data["x"]
coral_y = coral_data["y"]

['gml_id', 'lokalId', 'navnerom', 'verifiseringsdato', 'produkt', 'versjon', 'målemetode', 'nøyaktighet', 'medium', 'opphav', 'minimumsdybde', 'maksimumsdybde', 'naturtype', 'naturtypeNavn', 'kildeNavn', 'observasjonsMetode', 'observasjonsSted', 'observasjonsSlutt', 'lengdegrad', 'breddegrad', 'geometry', 'x', 'y']


In [ ]:
#Eddies
#Okubo-Weiss parameter
u = phys_states_2019["u_velocity"] 
v = phys_states_2019["v_velocity"]  
xc = phys_states_2019["xc"] 
yc = phys_states_2019["yc"] 

#grid spacing
dx = np.gradient(xc)  
dy = np.gradient(yc)  
dx = np.expand_dims(dx, axis=(0, 1, -1))  # Shape: (1, 1, 1, 950)

# dy should be broadcast to match the third dimension of u/v (555)
dy = np.expand_dims(dy, axis=(0, 1, -2))  # Shape: (1, 1, 555, 1)

du_dx_temp = np.gradient(u, axis=-1)  
du_dy_temp = np.gradient(u, axis=-2)  
dv_dx_temp = np.gradient(v, axis=-1) 
dv_dy_temp = np.gradient(v, axis=-2)  

du_dx = du_dx_temp / dx  
du_dy = du_dy_temp / dy  
dv_dx = dv_dx_temp / dx 
dv_dy = dv_dy_temp / dy  

#Okubo-Weiss components
zeta = dv_dx - du_dy  #relative vorticity
Sn = du_dx - dv_dy  #normal strain
Ss = dv_dx + du_dy  #shear strain

#Okubo-Weiss parameter
W = Sn**2 + Ss**2 - zeta**2

coral_W_time_series = []

for t in range(W.shape[0]):  
    W_t = W[t, 0, :, :]  #surface layer at time t

    interp_W = RegularGridInterpolator((yc, xc), W_t, bounds_error=False, fill_value=np.nan)

    #extract Okubo-Weiss values at coral locations
    coral_W_t = interp_W(np.array([coral_y, coral_x]).T)

    coral_W_time_series.append(coral_W_t)

coral_W_time_series = np.array(coral_W_time_series)


In [ ]:
"""
#Eddies correspond to negative W regions. 
W_std = W.std()
W_c = 0.2 * W_std  #eddy threshold
eddy_mask = W < -W_c  #identify eddies
"""

In [38]:
print(f"Shape of u: {u.shape}")
print(f"Shape of dx: {dx.shape}")
print(f"Shape of dy: {dy.shape}")

Shape of u: (365, 50, 555, 950)
Shape of dx: (1, 1, 950, 1)
Shape of dy: (1, 1, 1, 555)
